# EfficientNet 기반 음식 분류

**EfficientNet 모델**을 활용하여 음식 이미지를 분류하는 과정이다.
데이터 전처리, 모델 정의, 학습 및 평가 단계를 포함하며, 이미지 분류 정확도를 향상시키기 위한 다양한 실험을 진행된다.

- **주요 내용**  
  - 데이터 로드 및 전처리  
  - EfficientNet 모델 불러오기 (Pretrained)  
  - 분류기 학습 (Fine-tuning)  
  - 성능 평가 및 결과 시각화  


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# 원하는 루트 경로 (여기에 train/val 구조 만들기)
root_dir = "/content/dataset_split_food20_2"
os.makedirs(root_dir, exist_ok=True)

### Step 1. 데이터셋 학습/검증 분할

- `food_classes` 리스트에 20개 음식 카테고리를 정의
- 학습 데이터와 검증 데이터는 **8:2 비율**로 랜덤 분할



In [3]:
import zipfile
import shutil
import random

# 음식 클래스 목록
food_classes = [
    "감자전","감자탕","과메기","떡볶이","라면","만두","보쌈","삼계탕",
    "새우튀김","수제비","순대","양념치킨","육회","족발","짜장면",
    "짬뽕","콩국수","파전","피자","후라이드치킨"
]

# 구글드라이브 zip 파일 경로
zip_root = "/content/drive/MyDrive/"   # 네가 zip 올려둔 폴더

for cname in food_classes:
    zip_path = os.path.join(zip_root, f"{cname}.zip")
    extract_path = os.path.join(root_dir, cname)
    os.makedirs(extract_path, exist_ok=True)

    # 압축 해제
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    # 이미지 파일 모으기
    imgs = []
    for root, _, files in os.walk(extract_path):
        for f in files:
            if f.lower().endswith((".jpg",".jpeg",".png")):
                imgs.append(os.path.join(root,f))

    # train/val 디렉토리 생성
    train_out = os.path.join(root_dir, "train", cname)
    val_out   = os.path.join(root_dir, "val", cname)
    os.makedirs(train_out, exist_ok=True)
    os.makedirs(val_out, exist_ok=True)

    # train:val = 8:2 split
    random.shuffle(imgs)
    split_idx = int(len(imgs) * 0.8)
    train_imgs, val_imgs = imgs[:split_idx], imgs[split_idx:]

    # 파일 이동
    for src in train_imgs:
        shutil.move(src, os.path.join(train_out, os.path.basename(src)))
    for src in val_imgs:
        shutil.move(src, os.path.join(val_out, os.path.basename(src)))

    # 원래 압축 풀린 임시 폴더 제거
    shutil.rmtree(extract_path, ignore_errors=True)

print("✅ ImageFolder 구조 세팅 완료:", root_dir)


✅ ImageFolder 구조 세팅 완료: /content/dataset_split_food20_2


In [4]:
from torchvision import datasets, transforms

train_dataset = datasets.ImageFolder(os.path.join(root_dir, "train"))
val_dataset   = datasets.ImageFolder(os.path.join(root_dir, "val"))

print("클래스:", train_dataset.classes)
print("train 샘플 수:", len(train_dataset))
print("val 샘플 수:", len(val_dataset))


클래스: ['감자전', '감자탕', '과메기', '떡볶이', '라면', '만두', '보쌈', '삼계탕', '새우튀김', '수제비', '순대', '양념치킨', '육회', '족발', '짜장면', '짬뽕', '콩국수', '파전', '피자', '후라이드치킨']
train 샘플 수: 31999
val 샘플 수: 8009


### step2. Food-20 통합 파이프라인 (`food20_full.py`)

학습:**음식 이미지 분류 + 검증**을 위한 전체 파이프라인 제공

- **Utils/전처리**: 데이터 변환, 디바이스 설정, 폴더 생성  
- **Feature Extractor & AE**: EfficientNet-B4 특징 벡터 추출 → 클래스별 AutoEncoder 학습/검증  
- **Classifier**: EfficientNet-B4 기반 다중 분류기 학습 및 추론 (Top-1, Top-k)  
- **Classifier + AE**: 분류기 예측 후 AE로 재검증하여 신뢰도 향상  
- **Demo**: 간단한 2-클래스 학습 예시 포함  

-> **분류 정확도 + 신뢰성**을 동시에 높이는 구조


In [5]:
%%writefile /content/food20_full.py
# -*- coding: utf-8 -*-
"""
Food-20 Unified Pipeline (Loop-ready: AE + Classifier)

이 파일은 다음을 지원
1) 20-클래스(또는 N-클래스) 이미지 분류기 학습/추론 (사진 -> 어떤 음식인지)
2) 다수 클래스용 원-클래스 AE 반복 학습 (클래스별 '맞는지' 검증용)
3) 복합 추론: 분류기 top-k 후보 + 각 후보 AE 재검증(선택)

데이터 구조 (ImageFolder)
root/
  train/
    ramen/
    jjajang/
    bibimbap/
    ...
  val/
    ramen/
    jjajang/
    bibimbap/
    ...
"""

import os, json
from typing import List, Tuple, Dict, Optional

import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms
import timm



# ---------------------
# Utils / Transforms
# ---------------------
def get_device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

def build_transforms(img_size: int = 380, is_train: bool = True):
    if is_train:
        return transforms.Compose([
            transforms.Resize(int(img_size * 1.15)),
            transforms.CenterCrop(img_size),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])
        ])
    else:
        return transforms.Compose([
            transforms.Resize(int(img_size * 1.15)),
            transforms.CenterCrop(img_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])
        ])

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

# ---------------------
# EfficientNet-B4 feature extractor (for AE)
# ---------------------
def create_effnet_b4_feature_extractor(device: torch.device):
    effnet = timm.create_model("efficientnet_b4", pretrained=True)
    feat_dim = effnet.classifier.in_features  # 1792
    effnet.classifier = nn.Identity()
    effnet.to(device).eval()
    return effnet, feat_dim

@torch.no_grad()
def extract_features(loader: DataLoader, effnet: nn.Module, device: torch.device):
    feats, labels = [], []
    for imgs, labs in loader:
        imgs = imgs.to(device)
        out = effnet(imgs)             # [B, 1792]
        feats.append(out.cpu())
        labels.append(labs)
    return torch.cat(feats), torch.cat(labels)

# ---------------------
# One-Class AE (per class)
# ---------------------
class FeatureAE(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int = 512, bottleneck_dim: int = 128):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, bottleneck_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(bottleneck_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )
    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

def _subset_class(dataset: datasets.ImageFolder, class_idx: int):
    idx = [i for i, (_, y) in enumerate(dataset.samples) if y == class_idx]
    return torch.utils.data.Subset(dataset, idx)

def train_oneclass_ae(
    root: str,
    class_name: str,
    img_size: int = 380,
    batch_size: int = 32,
    epochs: int = 10,
    lr: float = 1e-3,
    weights_dir: str = "weights",
) -> float:
    """
    해당 class_name의 이미지(훈련셋만)로 Feature-AE를 학습하고
    threshold(mean + 2*std)을 저장/반환.
    """
    device = get_device()
    ensure_dir(weights_dir)

    train_tf = build_transforms(img_size, True)
    train_ds = datasets.ImageFolder(os.path.join(root, "train"), transform=train_tf)

    if class_name not in train_ds.class_to_idx:
        raise ValueError(f"{class_name} not in {train_ds.classes}")
    class_idx = train_ds.class_to_idx[class_name]

    train_subset = _subset_class(train_ds, class_idx)
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=2)

    effnet, feat_dim = create_effnet_b4_feature_extractor(device)
    train_feats, _ = extract_features(train_loader, effnet, device)

    ae = FeatureAE(feat_dim, 1792, 256).to(device)
    crit = nn.MSELoss()
    opt  = optim.Adam(ae.parameters(), lr=lr)

    ds = torch.utils.data.TensorDataset(train_feats, train_feats)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=True)

    for ep in range(epochs):
        ae.train()
        run = 0.0
        for x, _ in dl:
            x = x.to(device)
            recon = ae(x)
            loss = crit(recon, x)
            opt.zero_grad(); loss.backward(); opt.step()
            run += loss.item() * x.size(0)
        print(f"[AE][{class_name}] Epoch {ep+1}/{epochs} | Loss {run/len(dl.dataset):.6f}")

    # threshold from train set
    ae.eval()
    with torch.no_grad():
        recon = ae(train_feats.to(device))
        errors = torch.mean((recon - train_feats.to(device))**2, dim=1).cpu().numpy()
    threshold = float(np.mean(errors) + 2*np.std(errors))

    torch.save({
        "model": ae.state_dict(),
        "feat_dim": feat_dim,
        "hidden_dim": 1792,
        "bottleneck_dim": 256
    }, os.path.join(weights_dir, f"ae_{class_name}.pth"))

    with open(os.path.join(weights_dir, f"ae_{class_name}_threshold.txt"), "w") as f:
        f.write(str(threshold))
    print(f"[AE][{class_name}] Saved. threshold={threshold:.6f}")
    return threshold

def train_many_oneclass_aes(
    root: str,
    classes: Optional[List[str]] = None,
    img_size: int = 380,
    batch_size: int = 32,
    epochs: int = 10,
    lr: float = 1e-3,
    weights_dir: str = "weights",
) -> Dict[str, float]:
    """
    여러 클래스를 반복문으로 AE 일괄 학습. classes=None이면 train의 모든 클래스 자동.
    """
    train_ds = datasets.ImageFolder(os.path.join(root, "train"))
    target_classes = classes or train_ds.classes
    print(f"[AE] Target classes: {target_classes}")
    out = {}
    for cname in target_classes:
        out[cname] = train_oneclass_ae(
            root=root, class_name=cname, img_size=img_size,
            batch_size=batch_size, epochs=epochs, lr=lr, weights_dir=weights_dir
        )
    return out

@torch.no_grad()
def is_food_by_ae(
    img_path: str,
    class_name: str,
    img_size: int = 380,
    weights_dir: str = "weights",
) -> Tuple[bool, float, float]:
    """
    해당 class_name의 AE로 재구성오차를 측정, threshold와 비교.
    return: (is_class, recon_error, threshold)
    """
    device = get_device()
    w_path = os.path.join(weights_dir, f"ae_{class_name}.pth")
    t_path = os.path.join(weights_dir, f"ae_{class_name}_threshold.txt")
    if not (os.path.isfile(w_path) and os.path.isfile(t_path)):
        raise FileNotFoundError(f"Missing AE for {class_name}")

    ckpt = torch.load(w_path, map_location=device)
    ae = FeatureAE(
        input_dim=ckpt["feat_dim"],
        hidden_dim=ckpt["hidden_dim"],
        bottleneck_dim=ckpt["bottleneck_dim"]
    ).to(device)
    ae.load_state_dict(ckpt["model"])
    ae.eval()


    with open(t_path, "r") as f:
        threshold = float(f.read().strip())

    effnet, _ = create_effnet_b4_feature_extractor(device)
    tf = build_transforms(img_size, False)
    x = tf(Image.open(img_path).convert("RGB")).unsqueeze(0).to(device)
    feat  = effnet(x)
    recon = ae(feat)
    err = torch.mean((recon - feat)**2).item()
    return (err < threshold), float(err), float(threshold)

# ---------------------
# Multi-Class Classifier (사진 -> 무엇인지)
# ---------------------
class LabelSmoothingCE(nn.Module):
    def __init__(self, eps: float = 0.0):
        super().__init__()
        self.eps = eps
        self.log_softmax = nn.LogSoftmax(dim=1)
    def forward(self, logits, target):
        if self.eps == 0.0:
            return nn.functional.cross_entropy(logits, target)
        n = logits.size(1)
        log_probs = self.log_softmax(logits)
        with torch.no_grad():
            true_dist = torch.zeros_like(logits)
            true_dist.fill_(self.eps / (n - 1))
            true_dist.scatter_(1, target.unsqueeze(1), 1.0 - self.eps)
        return torch.mean(torch.sum(-true_dist * log_probs, dim=1))

def _maybe_weighted_sampler(ds: datasets.ImageFolder):
    counts = np.bincount([y for _, y in ds.samples], minlength=len(ds.classes))
    if (counts == 0).any():  # 방어
        return None
    weights = 1.0 / (counts + 1e-9)
    sw = [weights[y] for _, y in ds.samples]
    return WeightedRandomSampler(sw, num_samples=len(sw), replacement=True)

def _save_labelmap(ds: datasets.ImageFolder, out_path: str):
    labelmap = {int(v): k for k, v in ds.class_to_idx.items()}
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(labelmap, f, ensure_ascii=False, indent=2)

def _load_labelmap(path: str) -> Dict[int, str]:
    with open(path, "r", encoding="utf-8") as f:
        return {int(k): v for k, v in json.load(f).items()}

def accuracy_topk(logits: torch.Tensor, target: torch.Tensor, topk=(1,)):
    maxk = max(topk)
    _, pred = logits.topk(maxk, dim=1, largest=True, sorted=True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / target.size(0)))
    return res

def train_food_classifier(
    root: str,
    img_size: int = 380,
    batch_size: int = 32,
    epochs: int = 10,
    lr: float = 1e-4,
    label_smoothing: float = 0.0,
    freeze_backbone: bool = False,
    patience: int = 3,
    weights_dir: str = "weights",
):
    """
    N-클래스 음식 분류기 학습 (EfficientNet-B4 finetune). best 가중치 자동 저장.
    """
    device = get_device()
    ensure_dir(weights_dir)

    train_tf = build_transforms(img_size, True)
    val_tf   = build_transforms(img_size, False)

    train_ds = datasets.ImageFolder(os.path.join(root, "train"), transform=train_tf)
    val_ds   = datasets.ImageFolder(os.path.join(root, "val"), transform=val_tf)
    num_classes = len(train_ds.classes)
    print(f"[CLS] Classes({num_classes}): {train_ds.classes}")

    labelmap_path = os.path.join(weights_dir, "food_labelmap.json")
    _save_labelmap(train_ds, labelmap_path)
    print(f"[CLS] Saved label map -> {labelmap_path}")

    sampler = _maybe_weighted_sampler(train_ds)
    if sampler is None:
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
    else:
        train_loader = DataLoader(train_ds, batch_size=batch_size, sampler=sampler, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2)

    model = timm.create_model("efficientnet_b4", pretrained=True, num_classes=num_classes)
    if freeze_backbone:
        for name, p in model.named_parameters():
            if "classifier" not in name:
                p.requires_grad = False
    model.to(device)

    criterion = LabelSmoothingCE(eps=label_smoothing)
    optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max(1, epochs))

    best_acc, best_path = 0.0, os.path.join(weights_dir, "cls_food_best.pth")
    no_improve = 0

    for ep in range(epochs):
        model.train()
        tc = 0; tt = 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            tc += (out.argmax(1) == y).sum().item()
            tt += x.size(0)
        scheduler.step()
        train_acc = tc / max(1, tt)

        model.eval()
        vc = 0; vt = 0
        t1s, t3s = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                out = model(x)
                vc += (out.argmax(1) == y).sum().item()
                vt += x.size(0)
                t1, t3 = accuracy_topk(out, y, topk=(1,3))
                t1s.append(t1.item()); t3s.append(t3.item())
        val_acc  = vc / max(1, vt)
        val_t1   = float(np.mean(t1s)) if t1s else 0.0
        val_t3   = float(np.mean(t3s)) if t3s else 0.0
        print(f"[CLS] Epoch {ep+1:02d}/{epochs} | Train {train_acc:.4f} || Val {val_acc:.4f} Top1 {val_t1:.2f}% Top3 {val_t3:.2f}%")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save({"model": model.state_dict(), "num_classes": num_classes}, best_path)
            print(f"[CLS] Saved best -> {best_path} (Acc {best_acc:.4f})")
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"[CLS] Early stopping (patience={patience})")
                break

    print(f"[CLS] Best Val Acc: {best_acc:.4f} | Weights: {best_path}")

@torch.no_grad()
def predict_food(
    img_path: str,
    model_path: str = "weights/cls_food_best.pth",
    labelmap_path: str = "weights/food_labelmap.json",
    img_size: int = 380,
    topk: int = 3
):
    """
    분류기 추론 (사진 -> top-1, top-k 확률).
    """
    device = get_device()
    if not os.path.isfile(model_path):
        raise FileNotFoundError(model_path)
    if not os.path.isfile(labelmap_path):
        raise FileNotFoundError(labelmap_path)

    labelmap = _load_labelmap(labelmap_path)
    num_classes = len(labelmap)

    model = timm.create_model("efficientnet_b4", pretrained=False, num_classes=num_classes).to(device)
    ckpt = torch.load(model_path, map_location=device)
    model.load_state_dict(ckpt["model"]); model.eval()

    tf = build_transforms(img_size, False)
    x = tf(Image.open(img_path).convert("RGB")).unsqueeze(0).to(device)

    logits = model(x)
    probs  = torch.softmax(logits, dim=1).squeeze(0).cpu().numpy()
    k = min(topk, num_classes)
    idx = np.argsort(-probs)[:k].tolist()
    topk_pairs = [(labelmap[i], float(probs[i])) for i in idx]
    return {"top1": topk_pairs[0], "topk": topk_pairs}

# ---------------------
# Classifier -> AE verify combo
# ---------------------
@torch.no_grad()
def predict_food_then_verify(
    img_path: str,
    model_path: str = "weights/cls_food_best.pth",
    labelmap_path: str = "weights/food_labelmap.json",
    img_size: int = 380,
    topk: int = 3,
    verify_with_ae: bool = True,
    weights_dir: str = "weights",
):
    """
    1) 분류기 top-k 후보 예측
    2) (옵션) 각 후보 클래스에 대해 AE 재구성오차로 검증
    """
    pred = predict_food(img_path, model_path, labelmap_path, img_size, topk)
    if not verify_with_ae:
        return {"pred": pred, "ae_checks": None}

    checks = []
    for cname, prob in pred["topk"]:
        try:
            ok, err, th = is_food_by_ae(img_path, cname, img_size, weights_dir)
            checks.append({"class": cname, "prob": prob, "ae_ok": bool(ok), "recon_error": err, "threshold": th})
        except FileNotFoundError:
            checks.append({"class": cname, "prob": prob, "ae_ok": None, "recon_error": None, "threshold": None})
    return {"pred": pred, "ae_checks": checks}

# ---------------------
#기존 2-클래스 데모도 유지
# ---------------------
def quick_two_class_demo(
    data_dir: str,
    img_size: int = 380,
    batch_size: int = 32,
    epochs: int = 3,
    lr: float = 1e-4,
):
    device = get_device()
    train_tf = build_transforms(img_size, True)
    val_tf   = build_transforms(img_size, False)

    train_ds = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=train_tf)
    val_ds   = datasets.ImageFolder(os.path.join(data_dir, "val"), transform=val_tf)
    assert len(train_ds.classes) == 2, "Need exactly 2 classes for this demo."

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2)

    model = timm.create_model("efficientnet_b4", pretrained=True, num_classes=2).to(device)
    crit = nn.CrossEntropyLoss()
    opt  = optim.AdamW(model.parameters(), lr=lr)

    for ep in range(epochs):
        model.train(); tc=0; tt=0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            opt.zero_grad(); out = model(x); loss = crit(out, y)
            loss.backward(); opt.step()
            tc += (out.argmax(1) == y).sum().item()
            tt += x.size(0)

        model.eval(); vc=0; vt=0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                out = model(x)
                vc += (out.argmax(1) == y).sum().item()
                vt += x.size(0)

        print(f"[2C] Epoch {ep+1}/{epochs} | Train {tc/max(1,tt):.4f} | Val {vc/max(1,vt):.4f}")
    print("[2C] Done.")


Writing /content/food20_full.py


In [6]:
import os, glob

ROOT = "/content/dataset_split_food20_2"
bad_files = []

for f in glob.glob(ROOT + "/**/._*", recursive=True):
    bad_files.append(f)
    os.remove(f)

print("✅ 제거된 Mac 리소스 포크 파일 수:", len(bad_files))


✅ 제거된 Mac 리소스 포크 파일 수: 20004


In [7]:
import sys, importlib

# food20_full이 이미 로드되어 있으면 강제로 리로드
if 'food20_full' in sys.modules:
    importlib.reload(sys.modules['food20_full'])
else:
    import food20_full

import food20_full as f  # 앞으로 f. 으로 호출할 거에요


### step3. 결과 출력
- 짜장면 이미지 데이터를 활용해 검증

In [8]:
import sys, importlib

# 모듈 리로드
if "food20_full" in sys.modules:
    importlib.reload(sys.modules["food20_full"])
else:
    import food20_full

import food20_full as f   # 앞으로 f. 으로 호출

# 1. 데이터 루트 경로
ROOT = "/content/dataset_split_food20_2"

# 2. 분류기 학습
print("🚀 Step 1. 20-class 분류기 학습 시작")
f.train_food_classifier(
    root=ROOT,
    epochs=5,
    lr=1e-4,
    label_smoothing=0.1,
    freeze_backbone=False
)

# 3. 클래스별 AE 학습
print("\n🚀 Step 2. 클래스별 AE 학습 시작")
thresholds = f.train_many_oneclass_aes(
    root=ROOT,
    epochs=3
)
print("클래스별 Thresholds:", thresholds)

# 4. 추론 테스트
print("\n🚀 Step 3. 추론 실행")
test_img = "/content/common-9.jpg"
result = f.predict_food_then_verify(
    img_path=test_img,
    model_path="weights/cls_food_best.pth",
    labelmap_path="weights/food_labelmap.json",
    img_size=380,
    topk=3,
    verify_with_ae=True
)

print("\n최종 결과:")
print(result)


🚀 Step 1. 20-class 분류기 학습 시작
[CLS] Classes(20): ['감자전', '감자탕', '과메기', '떡볶이', '라면', '만두', '보쌈', '삼계탕', '새우튀김', '수제비', '순대', '양념치킨', '육회', '족발', '짜장면', '짬뽕', '콩국수', '파전', '피자', '후라이드치킨']
[CLS] Saved label map -> weights/food_labelmap.json


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/77.9M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


[CLS] Epoch 01/5 | Train 0.7035 || Val 0.8764 Top1 87.64% Top3 97.25%
[CLS] Saved best -> weights/cls_food_best.pth (Acc 0.8764)


...

[AE][족발] Epoch 1/3 | Loss 0.004970
[AE][족발] Epoch 2/3 | Loss 0.003750
[AE][족발] Epoch 3/3 | Loss 0.003118
[AE][족발] Saved. threshold=0.004530
[AE][짜장면] Epoch 1/3 | Loss 0.005458
[AE][짜장면] Epoch 2/3 | Loss 0.003703
[AE][짜장면] Epoch 3/3 | Loss 0.003009
[AE][짜장면] Saved. threshold=0.004651
[AE][짬뽕] Epoch 1/3 | Loss 0.003411
[AE][짬뽕] Epoch 2/3 | Loss 0.002409
[AE][짬뽕] Epoch 3/3 | Loss 0.002082
[AE][짬뽕] Saved. threshold=0.003677
[AE][콩국수] Epoch 1/3 | Loss 0.004975
[AE][콩국수] Epoch 2/3 | Loss 0.003930
[AE][콩국수] Epoch 3/3 | Loss 0.003356
[AE][콩국수] Saved. threshold=0.005261
[AE][파전] Epoch 1/3 | Loss 0.004219
[AE][파전] Epoch 2/3 | Loss 0.003178
[AE][파전] Epoch 3/3 | Loss 0.002673
[AE][파전] Saved. threshold=0.004471


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


[AE][피자] Epoch 1/3 | Loss 0.002910
[AE][피자] Epoch 2/3 | Loss 0.002235
[AE][피자] Epoch 3/3 | Loss 0.001972
[AE][피자] Saved. threshold=0.004085


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


[AE][후라이드치킨] Epoch 1/3 | Loss 0.004538
[AE][후라이드치킨] Epoch 2/3 | Loss 0.003426
[AE][후라이드치킨] Epoch 3/3 | Loss 0.002987
[AE][후라이드치킨] Saved. threshold=0.004971
클래스별 Thresholds: {'감자전': 0.005619973875582218, '감자탕': 0.004669167101383209, '과메기': 0.006523739546537399, '떡볶이': 0.004529417492449284, '라면': 0.004837975837290287, '만두': 0.005870405584573746, '보쌈': 0.004900071769952774, '삼계탕': 0.004549554083496332, '새우튀김': 0.0057545071467757225, '수제비': 0.004315010737627745, '순대': 0.005451973527669907, '양념치킨': 0.004694024566560984, '육회': 0.005324722267687321, '족발': 0.004530000500380993, '짜장면': 0.00465080002322793, '짬뽕': 0.003676858264952898, '콩국수': 0.005260597914457321, '파전': 0.004470800515264273, '피자': 0.004085250664502382, '후라이드치킨': 0.0049706390127539635}

🚀 Step 3. 추론 실행

최종 결과:
{'pred': {'top1': ('짜장면', 0.8835874795913696), 'topk': [('짜장면', 0.8835874795913696), ('후라이드치킨', 0.012351472862064838), ('감자전', 0.010002527385950089)]}, 'ae_checks': [{'class': '짜장면', 'prob': 0.8835874795913696, 'ae_ok': True